In [1]:
import PIL
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.functional import F


import torchvision
from torchvision.ops import nms
from torchvision.ops import RoIPool

from torchvision.ops import boxes as box_ops

import pandas as pd
import os
from tqdm.notebook import trange, tqdm
import cv2

from model import CenterNet

import matplotlib.pyplot as plt

In [2]:
use_cuda = True
device = torch.device('cuda:4' if use_cuda else 'cpu')
x = torch.Tensor([0]).cuda(device)

In [3]:
# dataset的路径
path = '../data/car-object-detection/data/'

In [4]:
'''
    image file name: array([bbox1,bbox2])
'''
train_bbox_pd = pd.read_csv(path + 'train_solution_bounding_boxes.csv')
train_bbox_np = train_bbox_pd.to_numpy()
train_bbox = {}
for d in train_bbox_np:
    if d[0] not in train_bbox:
        train_bbox.update({d[0]:[d[1:]]})
    else:
        train_bbox[d[0]].append(d[1:])
train_bbox = {d:np.array(train_bbox[d]) for d in train_bbox}

In [5]:
# dataset的路径
path = '../data/car-object-detection/data/'
train_imagefile = [i for i in os.listdir(path + 'training_images') if i[-3:] == 'jpg']
test_imagefile = [i for i in os.listdir(path + 'testing_images') if i[-3:] == 'jpg']

In [6]:
class cardset(torch.utils.data.Dataset):
    def __init__(self, path = '../data/car-object-detection/data/'):
        super(cardset, self).__init__()
        self.path = path
        # 读取图像文件名
        self.train_imagefile = [i for i in os.listdir(path + 'training_images') if i[-3:] == 'jpg']
        self.test_imagefile = [i for i in os.listdir(path + 'testing_images') if i[-3:] == 'jpg']
        self.train_img = []
        self.test_img = []
        # 读取训练集的bbox
        train_bbox_pd = pd.read_csv(self.path + 'train_solution_bounding_boxes.csv')
        self.train_bbox_np = train_bbox_pd.to_numpy()
        self.train_bbox = {}
        self.idx2file = {}
        self.file2idx = {}
        i = 0
        for d in self.train_bbox_np:
            if d[0] not in self.train_bbox:
                self.train_bbox.update({d[0]:[d[1:]]})
                self.idx2file.update({i:d[0]})
                self.file2idx.update({d[0]:i})
                i += 1
            else:
                self.train_bbox[d[0]].append(d[1:])
        self.train_bbox = {d:np.array(self.train_bbox[d],dtype = np.float32) for d in self.train_bbox}
        # 读取数据到内存
        for filename in tqdm(self.train_imagefile,desc = 'Reading train data'):
            img = Image.open(path + 'training_images/' + filename)
            self.train_img.append([filename,img])
            if filename not in self.train_bbox:
                self.train_bbox.update({filename:[]})
                self.idx2file.update({i:filename})
                self.file2idx.update({filename:i})
                i += 1
                
        for filename in tqdm(self.test_imagefile,desc = 'Reading test data'):
            img = Image.open(path + 'testing_images/' + filename)
            self.test_img.append([filename,img])
            
    def __getitem__(self, index):
        if isinstance(self.train_img[index][1],(Image.Image)):
            self.train_img[index][1] = torchvision.transforms.functional.pil_to_tensor(self.train_img[index][1])/255
        img = self.train_img[index][1]
#         print(self.train_bbox[self.train_img[index][0]])
        label_num = self.file2idx[self.train_img[index][0]]
        return img, label_num
    
    def __len__(self):
        return len(self.train_img)

In [7]:
car = cardset()

Reading train data:   0%|          | 0/1001 [00:00<?, ?it/s]

Reading test data:   0%|          | 0/175 [00:00<?, ?it/s]

In [8]:
def collect(batch):
    img,label_num = [i for i in zip(*batch)]
    img = torch.stack(img,0)
    label_num = torch.Tensor(label_num)
    return img,label_num
car_dataloader = torch.utils.data.DataLoader(car, batch_size = 8, shuffle = True,collate_fn = collect,drop_last = True)

In [9]:
# myiter = tqdm(car_dataloader,colour = '#0066FF')
# myiter.set_description_str('car dataloader')
# for x,y in myiter:
#     pass

In [10]:
def getTarget(x,y):
    target = []
    for i in y:
        t = car.train_bbox[car.idx2file[int(i)]]
        shape = len(t)
        label = 1
        if len(t) == 0:
            t = [[0,0,x.shape[-1]-1,x.shape[-2]-1]]
            shape = 1
            label = 0
        target.append({'bboxes':torch.Tensor(t).cuda(device), 'classes': torch.from_numpy(np.zeros(shape, dtype = np.int64) + label).cuda(device)})
    return target

In [11]:
# target = getTarget(x,y)

# model

In [12]:
centerNet = CenterNet(2)
centerNet.to(device)
centerNet.mode

'train'

In [13]:
x = x.cuda(device)

In [14]:
# result, losses = centerNet(x,target)

In [15]:
centerNet = torch.load('model/centerNet2.pt')
centerNet.to(device)
optimizer = optim.Adam(centerNet.parameters(),5e-4)

In [16]:
centerNet.train()
centerNet.mode = 'train'
min_loss = 1e9
for epoch in trange(70):
    myiter = tqdm(car_dataloader,colour = '#0066FF')
    myiter.set_description_str('car dataloader')
    all_loss = 0
    for x,y in myiter:
        x = x.cuda(device)
        target = getTarget(x,y)
        result, losses = centerNet(x,target)

        optimizer.zero_grad()
        loss = losses['point_focal_loss'] + 0.1*losses['size_loss'] + losses['offset_loss']
        loss.backward()
        optimizer.step()
        all_loss += float(loss)
        myiter.set_postfix(epoch = epoch,loss = float(loss),focal_loss = float(losses['point_focal_loss']),
                           size_loss = float(losses['size_loss']),
                           offset_loss = float(losses['offset_loss']),all_loss = all_loss)
        
        
    if all_loss < min_loss:
        min_loss = all_loss
        torch.save(centerNet, 'model/centerNet2.pt')
        plt.imshow(losses['heatmap'][0][1].detach().cpu())
        plt.show()
        plt.imshow(losses['gt_hm'][0][1].detach().cpu())
        plt.show()

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [17]:
class focal_loss(nn.Module):
    def __init__(self, alpha = 2, beta = 4, reduction = 'sum', N = 1):
        super(focal_loss, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.reduction = 'sum'
        self.N = N
        
    def forward(self, pred_hm, gt_hm):
        focalLoss = 0
        pred_hm = torch.clamp(pred_hm, 1e-6, 1-1e-6)
        pos_idx = (gt_hm == 1)
        neg_idx = (gt_hm != 1)
        # positive focal loss
        focalLoss += torch.sum((1 - pred_hm[pos_idx]).pow(self.alpha)*torch.log(pred_hm[pos_idx]))
        # negative
        focalLoss += torch.sum((1 - gt_hm[neg_idx]).pow(self.beta)*pred_hm[neg_idx].pow(self.alpha)*torch.log(1 - pred_hm[neg_idx]))
        if self.reduction == 'sum':
            return -focalLoss
        else:
            return -focalLoss/self.N

In [18]:
x,y = iter(car_dataloader).next()
x = x.cuda(device)
target = getTarget(x,y)
target

[{'bboxes': tensor([[  0.,   0., 675., 379.]], device='cuda:4'),
  'classes': tensor([0], device='cuda:4')},
 {'bboxes': tensor([[  0.,   0., 675., 379.]], device='cuda:4'),
  'classes': tensor([0], device='cuda:4')},
 {'bboxes': tensor([[  0.,   0., 675., 379.]], device='cuda:4'),
  'classes': tensor([0], device='cuda:4')},
 {'bboxes': tensor([[  0.0000, 183.6116,  86.5789, 246.7005],
          [185.3864, 178.7210, 305.2272, 236.9192],
          [548.8220, 181.1664, 676.0000, 244.0412]], device='cuda:4'),
  'classes': tensor([1, 1, 1], device='cuda:4')},
 {'bboxes': tensor([[207.3980, 190.9476, 237.7250, 212.9553]], device='cuda:4'),
  'classes': tensor([1], device='cuda:4')},
 {'bboxes': tensor([[  0.,   0., 675., 379.]], device='cuda:4'),
  'classes': tensor([0], device='cuda:4')},
 {'bboxes': tensor([[  0.,   0., 675., 379.]], device='cuda:4'),
  'classes': tensor([0], device='cuda:4')},
 {'bboxes': tensor([[  0.,   0., 675., 379.]], device='cuda:4'),
  'classes': tensor([0], devic

In [19]:
centerNet = CenterNet(2)
centerNet.to(device)
centerNet.mode
x = x[3:4].cuda(device)
target = getTarget(x,y[3:4])

In [20]:
target

[{'bboxes': tensor([[  0.0000, 183.6116,  86.5789, 246.7005],
          [185.3864, 178.7210, 305.2272, 236.9192],
          [548.8220, 181.1664, 676.0000, 244.0412]], device='cuda:4'),
  'classes': tensor([1, 1, 1], device='cuda:4')}]

In [21]:
fl = focal_loss(2,4,reduction='mean',N = 3)

In [22]:
optimizer = optim.AdamW(centerNet.parameters(),5e-4)

In [23]:
losses['pred_sz']

KeyError: 'pred_sz'

In [24]:
losses['size_loss']

tensor(250.0818, device='cuda:4', grad_fn=<DivBackward0>)

In [28]:
for i in range(50):
    result, losses = centerNet(x,target)
#     pred_hm = losses['heatmap']
#     gt_hm = losses['gt_hm']
#     pred_sz = losses['pred_sz']
#     gt_sz = losses['gt_sz']
#     loss = fl(pred_hm, gt_hm)

#     loss = F.mse_loss(pred_hm,gt_hm)
    loss = losses['size_loss']
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print(loss)
# print(pred_sz)
# print(gt_sz)

tensor(16.0546, device='cuda:4', grad_fn=<DivBackward0>)


In [ ]:
result[5]['scores'].argmax(),result[5]['scores'].max()

In [ ]:
result[5]['bboxes'][result[5]['scores']>0.2]

In [ ]:
target[5]

In [ ]:
plt.imshow(losses['heatmap'][5][1].detach().cpu())

In [ ]:
plt.imshow(losses['gt_hm'][5][1].detach().cpu())

In [ ]:
losses

In [ ]:
centerNet.eval()
centerNet.mode = 'test'
y,l = centerNet(x)

In [ ]:
y[1]['scores']

In [ ]:
y[2]['scores'].argmax()

In [ ]:
y[2]['scores'][y[2]['scores'].argmax()]

In [ ]:
y[2]['bboxes'][1764]

In [ ]:
y[2]

In [ ]:
target

In [ ]:
result[0]['bboxes'][:,[1,0]]

In [ ]:
torch.mean(losses['heatmap'])